# DSO110 - Final Group Project - Lottery
Alberta "Albi" Kovatcheva and Barbra Treston



## Background
Albi and Barbra have chosen the “Mega Millions Winning Numbers” dataset because the lottery is something that is familiar and accessible to a wide range of people worldwide; it would be difficult to find someone who hasn’t dreamed of hitting the jackpot and changing their life forever. However, it is also widely accepted that the lottery is not set up to favor the player.  In the case of Mega Millions, although there is a 1 in 24 chance of winning something, the odds of choosing all 6 numbers correctly to win the jackpot is 1 in 302,575,350 - a fact that is posted openly on both the New York Lottery and Mega Millions websites.  By analyzing the winning numbers data as well as complementary datasets on lottery retailers, lottery aid to local school districts, and monies recouped from the lottery winnings of public aid recipients, Albi and Barbra hope to glean insight to make actionable suggestions on how lottery players can get the best return on their investment as well as to demonstrate for the average person whether the lottery serves any societal good or whether it may be best to abstain from playing altogether. 



## Data Wrangling

### Import data.

In [171]:
import pandas as pd
import seaborn as sns
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
import numpy as np
from numpy import nan
import datetime as dt
from datetime import date

In [179]:
Winning_Numbers = pd.read_csv("Lottery_Mega_Millions_Winning_Numbers__Beginning_2002.csv")
pd.set_option("display.max_columns", None)
Winning_Numbers.head()

,Draw Date,Winning Numbers,Mega Ball,Multiplier
0,09/25/2020,20 36 37 48 67,16,2.0
1,09/29/2020,14 39 43 44 67,19,3.0
2,10/02/2020,09 38 47 49 68,25,2.0
3,10/06/2020,15 16 18 39 59,17,3.0
4,10/09/2020,05 11 25 27 64,13,2.0


In [173]:
Winners = pd.read_csv("jackpot winners.csv")
pd.set_option("display.max_columns", None)

Winners.head()

,Date,amount won,cash prize,location,state,gender
0,10/22/2021,108000000,75200000.0,lake havasu city AZ,AZ,Mixed
1,9/21/2021,431000000,314400000.0,Manhattan NY,NY,Unk
2,9/8/2021,55000000,38000000.0,crestwood IL,IL,Unk
3,5/21/2021,516000000,349300000.0,PA,PA,Unk
4,2/16/2021,96000000,68500000.0,Blossvale NY,NY,Mixed


In [174]:
Winners.rename(columns={'Date': 'Draw Date', ' amount  won ':'Jackpot'}, inplace=True)
Winners.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Draw Date   223 non-null    object 
 1   Jackpot     223 non-null    int64  
 2   cash prize  18 non-null     float64
 3   location    223 non-null    object 
 4   state       219 non-null    object 
 5   gender      222 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 10.6+ KB


### Extract month, day, year, weekday, and quarter from 'Draw Date'.

In [180]:
Winning_Numbers['month'] = pd.DatetimeIndex(Winning_Numbers['Draw Date']).month

In [181]:
Winning_Numbers['day'] = pd.DatetimeIndex(Winning_Numbers['Draw Date']).day

In [182]:
Winning_Numbers['year'] = pd.DatetimeIndex(Winning_Numbers['Draw Date']).year

In [183]:
Winning_Numbers['weekday'] = pd.DatetimeIndex(Winning_Numbers['Draw Date']).dayofweek
Winning_Numbers['weekday1'] = pd.DatetimeIndex(Winning_Numbers['Draw Date']).day_name()

In [184]:
Winning_Numbers['quarter'] = pd.DatetimeIndex(Winning_Numbers['Draw Date']).quarter
Winning_Numbers.head()


,Draw Date,Winning Numbers,Mega Ball,Multiplier,month,day,year,weekday,weekday1,quarter
0,09/25/2020,20 36 37 48 67,16,2.0,9,25,2020,4,Friday,3
1,09/29/2020,14 39 43 44 67,19,3.0,9,29,2020,1,Tuesday,3
2,10/02/2020,09 38 47 49 68,25,2.0,10,2,2020,4,Friday,4
3,10/06/2020,15 16 18 39 59,17,3.0,10,6,2020,1,Tuesday,4
4,10/09/2020,05 11 25 27 64,13,2.0,10,9,2020,4,Friday,4


In [185]:
Winning_Numbers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2036 entries, 0 to 2035
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Draw Date        2036 non-null   object 
 1   Winning Numbers  2036 non-null   object 
 2   Mega Ball        2036 non-null   int64  
 3   Multiplier       1133 non-null   float64
 4   month            2036 non-null   int64  
 5   day              2036 non-null   int64  
 6   year             2036 non-null   int64  
 7   weekday          2036 non-null   int64  
 8   weekday1         2036 non-null   object 
 9   quarter          2036 non-null   int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 159.2+ KB


### Convert 'Winning Numbers' to string and then separate terms into individual columns (5).

In [ ]:
Winning_Numbers["Winning Numbers"]= Winning_Numbers["Winning Numbers"].astype(str)

In [188]:
Winning_Numbers[['Ball1','Ball2',"Ball3","Ball4","Ball5"]] =  Winning_Numbers['Winning Numbers'].str.split(' ', expand=True)
Winning_Numbers.drop(columns =["Winning Numbers"], inplace = True)
Winning_Numbers.head()

,Draw Date,Mega Ball,Multiplier,month,day,year,weekday,weekday1,quarter,Ball1,Ball2,Ball3,Ball4,Ball5
0,09/25/2020,16,2.0,9,25,2020,4,Friday,3,20,36,37,48,67
1,09/29/2020,19,3.0,9,29,2020,1,Tuesday,3,14,39,43,44,67
2,10/02/2020,25,2.0,10,2,2020,4,Friday,4,09,38,47,49,68
3,10/06/2020,17,3.0,10,6,2020,1,Tuesday,4,15,16,18,39,59
4,10/09/2020,13,2.0,10,9,2020,4,Friday,4,05,11,25,27,64


In [189]:
Wins = pd.DataFrame(Winners.groupby('Draw Date')['Jackpot'].sum())
Wins

,Jackpot
Draw Date,
1/1/2008,33000000
1/1/2019,437000000
1/13/2009,22000000
1/16/2015,265000000
1/18/2005,131000000
...,...
9/30/2003,23300000
9/30/2011,113000000
9/6/2002,23300000


In [198]:
Winning_Numbers.index

Index(['09/25/2020', '09/29/2020', '10/02/2020', '10/06/2020', '10/09/2020',
       '10/13/2020', '10/16/2020', '10/20/2020', '10/23/2020', '10/27/2020',
       ...
       '10/26/2021', '10/29/2021', '11/02/2021', '11/05/2021', '11/09/2021',
       '11/12/2021', '11/16/2021', '11/19/2021', '11/23/2021', '11/26/2021'],
      dtype='object', name='Draw Date', length=2036)

In [199]:
Winning_Numbers1 = Winning_Numbers.join(Wins, how='left')

In [200]:
Winning_Numbers1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2036 entries, 09/25/2020 to 11/26/2021
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Mega Ball   2036 non-null   int64  
 1   Multiplier  1133 non-null   float64
 2   month       2036 non-null   int64  
 3   day         2036 non-null   int64  
 4   year        2036 non-null   int64  
 5   weekday     2036 non-null   int64  
 6   weekday1    2036 non-null   object 
 7   quarter     2036 non-null   int64  
 8   Ball1       2036 non-null   object 
 9   Ball2       2036 non-null   object 
 10  Ball3       2036 non-null   object 
 11  Ball4       2036 non-null   object 
 12  Ball5       2036 non-null   object 
 13  Jackpot     29 non-null     float64
dtypes: float64(2), int64(6), object(6)
memory usage: 303.1+ KB


In [201]:
Winning_Numbers1.describe()

,Mega Ball,Multiplier,month,day,year,weekday,quarter,Jackpot
count,2036.000000,1133.000000,2036.000000,2036.000000,2036.000000,2036.000000,2036.000000,2.900000e+01
mean,18.612475,3.369815,6.559921,15.726916,2011.638507,2.500000,2.521611,6.628103e+08
std,13.195995,1.014927,3.424417,8.802003,5.651637,1.500369,1.111488,2.831626e+09
min,1.000000,2.000000,1.000000,1.000000,2002.000000,1.000000,1.000000,2.330000e+07
25%,8.000000,3.000000,4.000000,8.000000,2007.000000,1.000000,2.000000,5.000000e+07
50%,15.000000,3.000000,7.000000,16.000000,2012.000000,2.500000,3.000000,8.800000e+07
75%,28.000000,4.000000,10.000000,23.000000,2017.000000,4.000000,4.000000,2.000000e+08
max,52.000000,5.000000,12.000000,31.000000,2021.000000,4.000000,4.000000,1.537000e+10


In [202]:
Winning_Numbers1['Ball1'] =Winning_Numbers1['Ball1'].astype('category')
Winning_Numbers1['Ball2'] =Winning_Numbers1['Ball2'].astype('category')
Winning_Numbers1['Ball3'] =Winning_Numbers1['Ball3'].astype('category')
Winning_Numbers1['Ball4'] =Winning_Numbers1['Ball4'].astype('category')
Winning_Numbers1['Ball5'] =Winning_Numbers1['Ball5'].astype('category')

cat_feat = ['Mega Ball','Multiplier']
for feat in cat_feat:
    Winning_Numbers1[feat] = Winning_Numbers1[feat].astype('category')

### Export data to excel file.

In [204]:
Winning_Numbers1.to_csv('Lottery_Mega_Millions_Winning_Numbers__Beginning_2002_Wrangled.csv')  